Librerias

In [1]:
import pred_lgbm as pred
import funciones_lgbm as f_lgbm
import pandas as pd
import numpy as np
import matplotlib as plt

e:\PWC-Challenge\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Entrenamos con una optimización de hiperparámetros utilizando optuna

In [2]:
def train_model_lgbm_optuna(data, test_size=0.2, random_state=42, n_trials=100, timeout=300):
    """
    Args:
        data: DataFrame completo con todas las columnas (incluyendo Salary)
        test_size: Proporción del conjunto de prueba
        random_state: Semilla aleatoria
        n_trials: Número máximo de pruebas de Optuna
        timeout: Tiempo límite en segundos para la optimización
    """
    print(f"\n🚀 Entrenando modelo LightGBM con Optuna + Features Estadísticos")
    print(f"   Trials: {n_trials}, Timeout: {timeout}s")
    
    # Importaciones necesarias
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    import lightgbm as lgb
    import optuna
    import numpy as np
    import warnings
    
    # Silenciar warnings
    warnings.filterwarnings('ignore')
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    
    # ============= PASO 1: PREPARAR GROUPING INFO =============
    print("📊 Paso 1: Preparando información de agrupación...")
    data_with_groups, grouping_info = f_lgbm.create_and_save_grouping_info(data)
    all_job_cats, all_seniority_cats = f_lgbm.get_all_categories(data_with_groups)
    
    print(f"   ✅ Grupos creados: Age_group, Exp_group")
    print(f"   ✅ Job categories: {len(all_job_cats)}")
    print(f"   ✅ Seniority levels: {len(all_seniority_cats)}")
    
    # ============= PASO 2: SEPARAR TARGET Y FEATURES =============
    
    print("🔄 Paso 2: Separando target y features...")
    
    X_data = data_with_groups.drop('Salary', axis=1)  # Variables disponibles en producción
    y = data_with_groups['Salary']  # Target
    
    print(f"   📊 Datos originales: {X_data.shape}")
    print(f"   🎯 Target: {len(y)} registros")
    
    # ============= PASO 3: SPLIT PRINCIPAL TRAIN/TEST =============
    print("✂️  Paso 3: Split principal train/test...")
    X_train_base, X_test_base, y_train, y_test = train_test_split(
        X_data, y, test_size=test_size, random_state=random_state
    )
    
    print(f"   📈 Train: {X_train_base.shape[0]} registros")
    print(f"   📉 Test:  {X_test_base.shape[0]} registros")
    
    # ============= PASO 4: CREAR FEATURES CON ESTADÍSTICAS =============
    print("🔧 Paso 4: Creando features con estadísticas...")
    
    # Crear features en TRAIN (calcula estadísticas)
    X_train, feature_names, stats_dict = f_lgbm.create_features_with_stats(
        X_train_base,
        all_job_categories=all_job_cats,
        all_seniority_levels=all_seniority_cats,
        stats_dict=None,
        is_training=True
    )
    
    # Aplicar features a TEST (usa estadísticas de train)
    X_test, _ = f_lgbm.create_features_with_stats(
        X_test_base,
        all_job_categories=all_job_cats,
        all_seniority_levels=all_seniority_cats,
        stats_dict=stats_dict,
        is_training=False
    )
    
    print(f"   ✅ Features totales: {X_train.shape[1]}")
    print(f"   ✅ Train: {X_train.shape}")
    print(f"   ✅ Test:  {X_test.shape}")
    
    # ============= PASO 5: SPLIT PARA VALIDACIÓN DE OPTUNA =============
    print("🔄 Paso 5: Split para validación de Optuna...")
    X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(
        X_train, y_train, test_size=0.2, random_state=random_state
    )
    
    print(f"   🎯 Train opt: {X_train_opt.shape}")
    print(f"   🔍 Validation: {X_val_opt.shape}")
    
    # ============= PASO 6: f_lgbmCIÓN OBJETIVO PARA OPTUNA =============
    def objective(trial):
        """f_lgbmción objetivo para Optuna"""
        
        # Hiperparámetros a optimizar
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'verbosity': -1,
            'random_state': random_state,
            'n_jobs': -1,
            
            # Parámetros a optimizar
            'num_leaves': trial.suggest_int('num_leaves', 10, 300),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 10.0, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'n_estimators': trial.suggest_int('n_estimators', 100, 2000)
        }
        
        # Crear y entrenar modelo
        model = lgb.LGBMRegressor(**params)
        
        try:
            # Entrenar con early stopping
            model.fit(
                X_train_opt, y_train_opt,
                eval_set=[(X_val_opt, y_val_opt)],
                callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
            )
            
            # Predecir en conjunto de validación
            y_pred = model.predict(X_val_opt)
            rmse = np.sqrt(mean_squared_error(y_val_opt, y_pred))
            
            return rmse
            
        except Exception as e:
            # Si hay error, devolver un valor alto
            return float('inf')
    
    # ============= PASO 7: OPTIMIZACIÓN CON OPTUNA =============
    print("🎯 Paso 7: Optimizando hiperparámetros con Optuna...")
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=random_state))
    
    # Optimizar
    study.optimize(objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True)
    
    print(f"   ✅ Optimización completada: {len(study.trials)} trials realizados")
    print(f"   🏆 Mejor RMSE de validación: ${study.best_value:,.2f}")
    
    # ============= PASO 8: MODELO FINAL =============
    print("🏆 Paso 8: Entrenando modelo final...")
    
    # Obtener mejores parámetros
    best_params = study.best_params.copy()
    best_params.update({
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'random_state': random_state,
        'n_jobs': -1
    })
    
    print("   📋 Mejores hiperparámetros encontrados:")
    for param, value in best_params.items():
        if param not in ['objective', 'metric', 'boosting_type', 'verbosity', 'random_state', 'n_jobs']:
            print(f"      {param}: {value}")
    
    # Entrenar modelo final con mejores parámetros
    final_model = lgb.LGBMRegressor(**best_params)
    
    try:
        # Entrenar en todo el conjunto de entrenamiento
        final_model.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
            callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
        )
        
        # ============= PASO 9: EVALUACIÓN FINAL =============
        print("📊 Paso 9: Evaluación final...")
        
        # Predicciones finales
        y_pred = final_model.predict(X_test)
        
        # Métricas en conjunto de prueba
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        
        # Cross-validation con el modelo optimizado
        print("   🔄 Realizando validación cruzada final...")
        cv_model = lgb.LGBMRegressor(**best_params)
        cv_scores = cross_val_score(
            cv_model, X_train, y_train, cv=5, 
            scoring='neg_mean_squared_error', n_jobs=-1
        )
        cv_rmse = np.sqrt(-cv_scores.mean())
        cv_std = np.sqrt(cv_scores.std())
        
        # ============= PASO 10: PREPARAR RESULTADOS =============
        print("📦 Paso 10: Preparando resultados finales...")
        
        # Resultados del modelo
        model_metrics = {
            'model': final_model,
            'rmse': rmse,
            'r2': r2,
            'mae': mae,
            'cv_rmse': cv_rmse,
            'cv_std': cv_std,
            'predictions': y_pred,
            'feature_importances': final_model.feature_importances_,
            'n_estimators_used': final_model.best_iteration_ if hasattr(final_model, 'best_iteration_') else final_model.n_estimators,
            'best_params': best_params,
            'optuna_study': study
        }
        
        # Resultado completo para compatibilidad
        final_results = {
            'model_results': {'LightGBM_Optuna': model_metrics},
            'best_model_name': 'LightGBM_Optuna',
            'best_model': final_model,
            'feature_names': feature_names,
            'job_categories': all_job_cats,
            'seniority_categories': all_seniority_cats,
            'stats_dict': stats_dict,
            'grouping_info': grouping_info,
            'X_test': X_test,
            'y_test': y_test,
            'X_train': X_train,
            'y_train': y_train,
            'optimization_study': study
        }
        
        # ============= MOSTRAR RESULTADOS =============
        print(f"\n🎉 RESULTADOS FINALES:")
        print(f"   RMSE: ${rmse:,.2f}")
        print(f"   R²: {r2:.3f}")
        print(f"   MAE: ${mae:,.2f}")
        print(f"   CV RMSE: ${cv_rmse:,.2f} (±{cv_std:,.2f})")
        print(f"   Features totales: {len(feature_names)}")
        print(f"   Estimadores utilizados: {model_metrics['n_estimators_used']}")
        print(f"   Mejora vs RMSE de validación: {((study.best_value - rmse) / study.best_value * 100):+.2f}%")
        
        return final_results
        
    except Exception as e:
        print(f"❌ Error entrenando modelo final: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return None

PIPELINE - LGBM

In [3]:
def pipeline_fe():
    print("🚀 INICIANDO PIPELINE")

    # 1 . Cargar datos
    data = pd.read_csv('../../../dataC/imputado.csv')
    data["Description"] = data["Description"].fillna("")
    
    data = data.dropna()
    # Se mejora levemente los errores realizando esta imputación y dejando los otros nulos de los otros features.
    
    # Entrenar modelo
    model_results =train_model_lgbm_optuna(data)

    # Analizar optimización
    best_params, best_value = f_lgbm.analyze_optuna_optimization(model_results['optimization_study'])

    # Visualizar proceso
    #plot_optuna_optimization(model_results['optimization_study'])

    # 5. Analizar importancia
    #feature_importance = fun.analyze_feature_importance(X,feature_names,model)

    # 6. Analizar predicciones
    #predictions_analysis = f.analyze_predictions(model_results)

    # 7. Comparar modelos
    #fun.create_comparison_chart(model_results)

    print("\n🎉 ANÁLISIS COMPLETADO!")
    print("="*50)

    # Resumen final
    best_name = model_results['best_model_name']
    best_result = model_results['model_results'][best_name]

    print(f"\n🏆 RESUMEN FINAL:")
    print(f"   Mejor modelo: {best_name}")
    print(f"   RMSE: ${best_result['rmse']:,.2f}")
    print(f"   R²: {best_result['r2']:.3f}")
    print(f"   CV RMSE: ${best_result['cv_rmse']:,.2f}")
    #print(f"   Características utilizadas: {len(feature_names)}")
    
    

     
    return model_results,data
    # ,model,X,feature_importance
    
results,data=pipeline_fe()   

🚀 INICIANDO PIPELINE

🚀 Entrenando modelo LightGBM con Optuna + Features Estadísticos
   Trials: 100, Timeout: 300s
📊 Paso 1: Preparando información de agrupación...
📊 Creando grupos y guardando información de rangos...
   ✅ Grupos creados: Age_group, Exp_group
   ✅ Job categories: 12
   ✅ Seniority levels: 5
🔄 Paso 2: Separando target y features...
   📊 Datos originales: (369, 9)
   🎯 Target: 369 registros
✂️  Paso 3: Split principal train/test...
   📈 Train: 295 registros
   📉 Test:  74 registros
🔧 Paso 4: Creando features con estadísticas...
🔧 Creando características completas para producción (originales + estadísticos)...
🔧 Creando todas las características mejoradas...
✅ Creadas 61 características en total
   - Variables numéricas básicas: 3
   - Variables de educación: 3
   - Variables de job category: 12
   - Variables de seniority: 6
   - Variables de texto: 4
   - Ratios y scores: 5
📊 Creando features estadísticos para producción (TRAIN)...
   🔄 Calculando estadísticas en TRAI

Best trial: 88. Best value: 13311.5: 100%|██████████| 100/100 [00:10<00:00,  9.35it/s, 10.69/300 seconds]


   ✅ Optimización completada: 100 trials realizados
   🏆 Mejor RMSE de validación: $13,311.46
🏆 Paso 8: Entrenando modelo final...
   📋 Mejores hiperparámetros encontrados:
      num_leaves: 21
      learning_rate: 0.1103186092201265
      feature_fraction: 0.906524091469967
      bagging_fraction: 0.6815973162203444
      bagging_freq: 1
      min_child_samples: 23
      min_child_weight: 0.7585553500711019
      reg_alpha: 9.248611683528368
      reg_lambda: 1.5875177712326718
      max_depth: 13
      n_estimators: 803
📊 Paso 9: Evaluación final...
   🔄 Realizando validación cruzada final...
📦 Paso 10: Preparando resultados finales...

🎉 RESULTADOS FINALES:
   RMSE: $16,963.52
   R²: 0.886
   MAE: $9,915.00
   CV RMSE: $11,777.75 (±6,188.65)
   Features totales: 93
   Estimadores utilizados: 134
   Mejora vs RMSE de validación: -27.44%
🔬 Análisis de optimización Optuna:
   Número total de trials: 100
   Mejor valor: $13,311.46
   Trials completados: 100
   Trials fallidos: 0
   🔍 Im

In [4]:
complete_package = f_lgbm.save_with_stats(
        results,
        filename="../../../modelos/salary_with_stats.pkl"
    )

💾 Guardando modelo completo con features estadísticos...
✅ Modelo completo guardado en ../../../modelos/salary_with_stats.pkl
📦 Incluye:
   🤖 Modelo LightGBM optimizado
   🔢 93 características
   📊 Features estadísticos (stats_dict)
   🏷️  12 categorías de trabajo
   👔 5 niveles de seniority
   📈 Información de agrupación
   📉 Métricas del modelo


Probamos con  una nueva predicción

In [6]:
import joblib

In [7]:
model_package = joblib.load("../../../modelos/salary_with_stats.pkl")
def test_prediction(model_package):
    """
    Test para verificar que la predicción funciona con un solo registro
    """
    print("🧪 Testing predicción con un solo registro...")
    
    # Crear registro de prueba
    exp_group, age_group = pred.calculate_groups(
    age=60, 
    years_of_experience=24, 
    grouping_info=model_package.get('grouping_info')
    )

    test_record = pd.DataFrame({
    'Age': [60],
    'Gender': ['Male'],
    'Education_Level': ["PhD"],
    'Job_Title': ['CEO'],
    'Years_of_Experience': [24],
    'Description': ['I work with machine learning models and data analysis'],
    'Exp_group': [exp_group],      # ← Calculado automáticamente
    'Age_group': [age_group]       # ← Calculado automáticamente
    })
    
    try:
        prediction = pred.predict(test_record, model_package)
        print(f"✅ Test exitoso: Predicción = ${prediction:,.2f}")
        return True
    except Exception as e:
        print(f"❌ Test falló: {e}")
        return False

In [ ]:
import joblib
model_package = joblib.load("../../../modelos/salary_with_stats.pkl")

In [ ]:
test_prediction(model_package)

🧪 Testing predicción con un solo registro...
🎯 Predicción con modelo completo (un solo registro)...
   🔢 Features esperadas: 93
🔧 Creando características completas para un solo registro...
🔧 Creando todas las características mejoradas...
✅ Creadas 61 características en total
   - Variables numéricas básicas: 3
   - Variables de educación: 3
   - Variables de job category: 12
   - Variables de seniority: 6
   - Variables de texto: 4
   - Ratios y scores: 5
📊 Creando features estadísticos para un solo registro...
   ✅ Creadas 32 features estadísticos para un solo registro
✅ Features totales para un solo registro: 93
   - Originales: 61
   - Estadísticos: 32
   🔢 Features generadas: 93
   💰 Predicción: $172,229.38
   ✅ Predicción exitosa con 93 features
✅ Test exitoso: Predicción = $172,229.38


True

Análisis de la optimización

In [ ]:


# 1. ANÁLISIS DE IMPORTANCIA DE HIPERPARÁMETROS
print("🔍 ANÁLISIS DE IMPORTANCIA DE HIPERPARÁMETROS")
print("="*50)

# Suponiendo que tienes tu study guardado en 'results' o 'study'
best_params, best_value = f_lgbm.analyze_optuna_optimization(results['optimization_study'])

# 2. ANÁLISIS DETALLADO DE MÉTRICAS FINALES
print("\n📊 MÉTRICAS FINALES DEL MODELO OPTIMIZADO")
print("="*50)

final_model = results['best_model']
model_results = results['model_results']['LightGBM_Optuna']

print(f"🎯 RMSE en Test: ${model_results['rmse']:,.2f}")
print(f"📈 R² Score: {model_results['r2']:.4f} ({model_results['r2']*100:.2f}%)")
print(f"📉 MAE: ${model_results['mae']:,.2f}")
print(f"🔄 CV RMSE: ${model_results['cv_rmse']:,.2f} (±{model_results['cv_std']:,.2f})")
print(f"🌳 Estimadores usados: {model_results['n_estimators_used']}")

# Calcular mejora respecto a baseline
rmse_improvement = ((12600 - model_results['rmse']) / 12600) * 100
print(f"⚡ Mejora total: {rmse_improvement:.2f}% respecto al inicio")

# 3. ANÁLISIS DE IMPORTANCIA DE CARACTERÍSTICAS
print("\n🔍 IMPORTANCIA DE CARACTERÍSTICAS")
print("="*50)

feature_importance_df = f_lgbm.get_feature_importance(
    final_model, 
    feature_names=results.get('feature_names'), 
    top_n=20
)

# 4. ANÁLISIS DE CALIDAD DE PREDICCIONES
print("\n🎯 ANÁLISIS DE CALIDAD DE PREDICCIONES")
print("="*50)

y_test = results['y_test']
y_pred = model_results['predictions']
residuos = y_test - y_pred

# Estadísticas de residuos
print(f"Media de residuos: ${np.mean(residuos):,.2f}")
print(f"Std de residuos: ${np.std(residuos):,.2f}")
print(f"Mediana abs residuos: ${np.median(np.abs(residuos)):,.2f}")

# Percentiles de error absoluto
abs_errors = np.abs(residuos)
print(f"\nPercentiles de error absoluto:")
print(f"  25%: ${np.percentile(abs_errors, 25):,.2f}")
print(f"  50%: ${np.percentile(abs_errors, 50):,.2f}")
print(f"  75%: ${np.percentile(abs_errors, 75):,.2f}")
print(f"  95%: ${np.percentile(abs_errors, 95):,.2f}")

# 5. ANÁLISIS DE OUTLIERS EN PREDICCIONES
print(f"\n🔍 ANÁLISIS DE OUTLIERS")
print("="*50)

# Definir outliers como errores > 2 std
threshold = 2 * np.std(abs_errors)
outliers = abs_errors > threshold
n_outliers = np.sum(outliers)
outlier_pct = (n_outliers / len(abs_errors)) * 100

print(f"Outliers detectados: {n_outliers} ({outlier_pct:.2f}%)")
print(f"Threshold usado: ${threshold:,.2f}")

if n_outliers > 0:
    print(f"Error promedio en outliers: ${np.mean(abs_errors[outliers]):,.2f}")
    print(f"Valor real promedio outliers: ${np.mean(y_test[outliers]):,.2f}")

# 6. ANÁLISIS DE HIPERPARÁMETROS ÓPTIMOS
print(f"\n⚙️ HIPERPARÁMETROS ÓPTIMOS ENCONTRADOS")
print("="*50)

important_params = [
    'num_leaves', 'learning_rate', 'max_depth', 'n_estimators',
    'feature_fraction', 'bagging_fraction', 'reg_alpha', 'reg_lambda'
]

for param in important_params:
    if param in best_params:
        print(f"{param:20s}: {best_params[param]}")

# 7. COMPARACIÓN CON CONFIGURACIÓN BASE
print(f"\n📈 COMPARACIÓN CON CONFIGURACIÓN BASE")
print("="*50)

# Configuración base típica
base_config = {
    'num_leaves': 31,  # Default
    'learning_rate': 0.1,  # Default
    'max_depth': -1,  # Default (sin límite)
    'n_estimators': 100,  # Default
    'feature_fraction': 1.0,  # Default
    'bagging_fraction': 1.0,  # Default
}

print("Parámetro             Base      Optimizado    Cambio")
print("-" * 55)
for param in important_params[:6]:  # Top 6 más importantes
    if param in best_params and param in base_config:
        base_val = base_config[param]
        opt_val = best_params[param]
        if isinstance(opt_val, float):
            change = f"{((opt_val - base_val) / base_val * 100):+.1f}%"
            print(f"{param:20s} {base_val:8.3f} {opt_val:11.3f} {change:>10s}")
        else:
            change = f"{opt_val - base_val:+d}"
            print(f"{param:20s} {base_val:8d} {opt_val:11d} {change:>10s}")

# 8. RECOMENDACIONES FINALES
print(f"\n💡 RECOMENDACIONES Y CONCLUSIONES")
print("="*50)

print("✅ Aspectos positivos:")
print(f"  • Convergencia estable alcanzada en ~45 trials")
print(f"  • Mejora significativa: {rmse_improvement:.1f}% en RMSE")
print(f"  • R² de {model_results['r2']:.3f} indica buen ajuste")
print(f"  • CV estable con baja varianza")

if model_results['r2'] > 0.8:
    print(f"  • Excelente capacidad predictiva (R² > 0.8)")
elif model_results['r2'] > 0.6:
    print(f"  • Buena capacidad predictiva (R² > 0.6)")

print(f"\n⚠️  Áreas de atención:")
if outlier_pct > 5:
    print(f"  • {outlier_pct:.1f}% de outliers - considerar análisis adicional")
if model_results['cv_std'] > model_results['cv_rmse'] * 0.1:
    print(f"  • Varianza en CV relativamente alta - validar estabilidad")



🔍 ANÁLISIS DE IMPORTANCIA DE HIPERPARÁMETROS
🔬 Análisis de optimización Optuna:
   Número total de trials: 100
   Mejor valor: $13,311.46
   Trials completados: 100
   Trials fallidos: 0
   🔍 Importancia de hiperparámetros (Top 10):
       1. min_child_samples    - 0.8167
       2. learning_rate        - 0.1355
       3. num_leaves           - 0.0151
       4. feature_fraction     - 0.0094
       5. reg_lambda           - 0.0074
       6. bagging_fraction     - 0.0059
       7. reg_alpha            - 0.0034
       8. max_depth            - 0.0031
       9. n_estimators         - 0.0027
      10. min_child_weight     - 0.0004

📊 MÉTRICAS FINALES DEL MODELO OPTIMIZADO
🎯 RMSE en Test: $16,963.52
📈 R² Score: 0.8859 (88.59%)
📉 MAE: $9,915.00
🔄 CV RMSE: $11,777.75 (±6,188.65)
🌳 Estimadores usados: 134
⚡ Mejora total: -34.63% respecto al inicio

🔍 IMPORTANCIA DE CARACTERÍSTICAS

🔍 Top 20 características más importantes:
    1. avg_word_length      - 59.0000
    2. description_length_zscore - 

El 25% de los casos se predicen con un error menor a $1250 ok.

El 50% (mediana) están dentro de $6000 ok.

Pero un 25% tiene errores mayores a $12.800, y un 5% supera los $24.500

Salarios de aproximadamente $200 000 un error 12%

Construir nuevas variables que puedan captar este comportamiento - work

El modelo muestra un excelente rendimiento general, con métricas sólidas y comportamiento estable. Revisar casos extremos (outliers) y si hay segmentos que afectan la varianza de la validación cruzada.

Posible forma en la que se generan los grupos


Revisemos los outliers de predicción

In [ ]:
# ANÁLISIS DE OUTLIERS USANDO SOLO MODEL_RESULTS
print("🔍 ANÁLISIS DE OUTLIERS CON MODEL_RESULTS")
print("="*60)

# 1. EXTRAER DATOS DESDE MODEL_RESULTS
model_results = results['model_results']['LightGBM_Optuna']
y_test = results['y_test']
y_pred = model_results['predictions']

print(f"📊 Datos disponibles:")
print(f"   • Predicciones: {len(y_pred)} casos")
print(f"   • Valores reales: {len(y_test)} casos")

# 2. CALCULAR ERRORES Y IDENTIFICAR OUTLIERS
residuos = y_test - y_pred
abs_errors = np.abs(residuos)
rel_errors = (abs_errors / y_test) * 100

# Convertir a arrays numpy para evitar problemas de indexing
y_test_array = y_test.values if hasattr(y_test, 'values') else np.array(y_test)
y_pred_array = np.array(y_pred)
abs_errors_array = np.abs(y_test_array - y_pred_array)
rel_errors_array = (abs_errors_array / y_test_array) * 100

# Threshold para outliers (2 std)
threshold = 2 * np.std(abs_errors_array)
outliers_mask = abs_errors_array > threshold
outliers_indices = np.where(outliers_mask)[0]

print(f"\n🎯 DETECCIÓN DE OUTLIERS:")
print(f"   • Threshold: ${threshold:,.2f}")
print(f"   • Outliers encontrados: {len(outliers_indices)} casos")
print(f"   • Porcentaje: {(len(outliers_indices)/len(y_test))*100:.2f}%")

# 3. ANÁLISIS DETALLADO DE OUTLIERS
print(f"\n🔴 TOP 15 OUTLIERS MÁS PROBLEMÁTICOS")
print("="*80)

# Crear DataFrame con la información de outliers
outliers_df = pd.DataFrame({
    'Index': outliers_indices,
    'Salary_Real': y_test_array[outliers_indices],
    'Salary_Pred': y_pred_array[outliers_indices],
    'Error_Abs': abs_errors_array[outliers_indices],
    'Error_Rel': rel_errors_array[outliers_indices]
})

# Ordenar por error absoluto (descendente)
outliers_df = outliers_df.sort_values('Error_Abs', ascending=False)

print("Rank  Index   Salario Real   Predicción    Error Abs    Error %")
print("-" * 75)

for i, (_, row) in enumerate(outliers_df.head(15).iterrows(), 1):
    print(f"{i:2d}    {row['Index']:4.0f}   ${row['Salary_Real']:9.0f}   ${row['Salary_Pred']:9.0f}   ${row['Error_Abs']:8.0f}   {row['Error_Rel']:6.1f}%")

# 4. ESTADÍSTICAS DE OUTLIERS vs NORMALES
print(f"\n📈 COMPARACIÓN: OUTLIERS vs CASOS NORMALES")
print("="*60)

normal_mask = ~outliers_mask

# Salarios reales
outlier_salaries = y_test_array[outliers_mask]
normal_salaries = y_test_array[normal_mask]

print("Métrica                 Outliers      Normales    Diferencia")
print("-" * 65)
print(f"{'Media Salario':20s}   ${outlier_salaries.mean():9.0f} ${normal_salaries.mean():9.0f}   ${outlier_salaries.mean() - normal_salaries.mean():+9.0f}")
#print(f"{'Mediana Salario':20s}   ${outlier_salaries.median():9.0f} ${normal_salaries.median():9.0f}   ${outlier_salaries.median() - normal_salaries.median():+9.0f}")
print(f"{'Std Salario':20s}   ${outlier_salaries.std():9.0f} ${normal_salaries.std():9.0f}   ${outlier_salaries.std() - normal_salaries.std():+9.0f}")

# Errores
outlier_errors = abs_errors_array[outliers_mask]
normal_errors = abs_errors_array[normal_mask]

print(f"{'Error Promedio':20s}   ${outlier_errors.mean():9.0f} ${normal_errors.mean():9.0f}   ${outlier_errors.mean() - normal_errors.mean():+9.0f}")
print(f"{'Error Mediano':20s}   ${np.median(outlier_errors):9.0f} ${np.median(normal_errors):9.0f}   ${np.median(outlier_errors) - np.median(normal_errors):+9.0f}")

# 5. ANÁLISIS DE DISTRIBUCIÓN DE OUTLIERS
print(f"\n📊 DISTRIBUCIÓN DE OUTLIERS POR RANGOS SALARIALES")
print("="*60)

# Definir rangos salariales
salary_ranges = [
    (0, 50000, "Bajo (<$50K)"),
    (50000, 100000, "Medio ($50K-$100K)"),
    (100000, 150000, "Alto ($100K-$150K)"),
    (150000, 200000, "Muy Alto ($150K-$200K)"),
    (200000, float('inf'), "Premium (>$200K)")
]

print("Rango Salarial        Total    Outliers   Tasa Outlier")
print("-" * 60)

for min_sal, max_sal, label in salary_ranges:
    # Casos en este rango
    in_range = (y_test_array >= min_sal) & (y_test_array < max_sal)
    total_in_range = in_range.sum()
    
    if total_in_range > 0:
        # Outliers en este rango
        outliers_in_range = (in_range & outliers_mask).sum()
        outlier_rate = (outliers_in_range / total_in_range) * 100
        
        print(f"{label:20s}   {total_in_range:5d}    {outliers_in_range:8d}   {outlier_rate:8.2f}%")

# 6. ANÁLISIS DE PATRONES EN PREDICCIONES
print(f"\n🎯 PATRONES EN LAS PREDICCIONES ERRÓNEAS")
print("="*60)

# Casos donde el modelo subestima mucho
underestimated = outliers_df[outliers_df['Salary_Real'] > outliers_df['Salary_Pred']]
underestimated_severe = underestimated[underestimated['Error_Rel'] > 30]

# Casos donde el modelo sobreestima mucho
overestimated = outliers_df[outliers_df['Salary_Real'] < outliers_df['Salary_Pred']]
overestimated_severe = overestimated[overestimated['Error_Rel'] > 30]

print(f"📉 Subestimaciones severas (>30%): {len(underestimated_severe)} casos")
if len(underestimated_severe) > 0:
    print(f"   • Error promedio: ${underestimated_severe['Error_Abs'].mean():,.0f}")
    print(f"   • Salario real promedio: ${underestimated_severe['Salary_Real'].mean():,.0f}")
    print(f"   • Predicción promedio: ${underestimated_severe['Salary_Pred'].mean():,.0f}")

print(f"\n📈 Sobreestimaciones severas (>30%): {len(overestimated_severe)} casos")
if len(overestimated_severe) > 0:
    print(f"   • Error promedio: ${overestimated_severe['Error_Abs'].mean():,.0f}")
    print(f"   • Salario real promedio: ${overestimated_severe['Salary_Real'].mean():,.0f}")
    print(f"   • Predicción promedio: ${overestimated_severe['Salary_Pred'].mean():,.0f}")

# 7. CASOS EXTREMOS PARA INVESTIGACIÓN MANUAL
print(f"\n🔍 CASOS EXTREMOS PARA INVESTIGACIÓN")
print("="*60)

# Casos con errores relativos extremos
extreme_cases = outliers_df[outliers_df['Error_Rel'] > 50]
print(f"⚠️  Casos con error relativo >50%: {len(extreme_cases)}")

if len(extreme_cases) > 0:
    print("\nCasos que requieren investigación manual:")
    for _, row in extreme_cases.head(5).iterrows():
        error_type = "Subestimó" if row['Salary_Real'] > row['Salary_Pred'] else "Sobreestimó"
        print(f"  • Índice {row['Index']:.0f}: {error_type}")
        print(f"    Real: ${row['Salary_Real']:,.0f} | Pred: ${row['Salary_Pred']:,.0f} | Error: {row['Error_Rel']:.1f}%")

# 8. MÉTRICAS DE OUTLIERS
print(f"\n📊 MÉTRICAS ESPECÍFICAS DE OUTLIERS")
print("="*60)

print(f"🎯 Estadísticas de errores en outliers:")
print(f"   • Error absoluto promedio: ${outliers_df['Error_Abs'].mean():,.2f}")
print(f"   • Error absoluto mediano: ${outliers_df['Error_Abs'].median():,.2f}")
print(f"   • Error relativo promedio: {outliers_df['Error_Rel'].mean():.2f}%")
print(f"   • Error relativo mediano: {outliers_df['Error_Rel'].median():.2f}%")

# Percentiles de errores en outliers
print(f"\n📈 Percentiles de error absoluto en outliers:")
print(f"   • P25: ${np.percentile(outliers_df['Error_Abs'], 25):,.0f}")
print(f"   • P50: ${np.percentile(outliers_df['Error_Abs'], 50):,.0f}")
print(f"   • P75: ${np.percentile(outliers_df['Error_Abs'], 75):,.0f}")
print(f"   • P90: ${np.percentile(outliers_df['Error_Abs'], 90):,.0f}")

# 9. RECOMENDACIONES BASADAS EN EL ANÁLISIS
print(f"\n💡 RECOMENDACIONES PARA MEJORAR EL MODELO")
print("="*60)

if len(extreme_cases) > 0:
    print("🔴 PRIORIDAD ALTA:")
    print(f"   • Investigar {len(extreme_cases)} casos con error >50%")
    print("   • Verificar posibles errores en los datos")
    print("   • Considerar exclusión temporal para validar")

if len(underestimated_severe) > len(overestimated_severe):
    print(f"\n📉 El modelo tiende a SUBESTIMAR salarios altos:")
    print("   • Revisar feature engineering para capturar mejor salarios premium")
    print("   • Considerar transformación logarítmica")
    print("   • Ajustar regularización para permitir predicciones más altas")
elif len(overestimated_severe) > len(underestimated_severe):
    print(f"\n📈 El modelo tiende a SOBREESTIMAR:")
    print("   • Aumentar regularización")
    print("   • Revisar outliers en datos de entrenamiento")

outlier_rate = (len(outliers_indices)/len(y_test))*100
if outlier_rate > 10:
    print(f"\n⚠️  Tasa de outliers alta ({outlier_rate:.2f}%):")
    print("   • Revisar calidad de los datos")
    print("   • Considerar ensemble de modelos")
    print("   • Implementar detección de anomalías en preprocessing")

print(f"\n✅ PRÓXIMOS PASOS:")
print("1. Exportar índices de outliers para análisis manual")
print("2. Verificar datos originales de casos extremos")
print("3. Implementar mejoras específicas identificadas")
print("4. Re-evaluar modelo sin outliers extremos")

# 10. EXPORTAR OUTLIERS PARA ANÁLISIS
print(f"\n💾 DATOS DE OUTLIERS PARA EXPORTAR:")
print("="*60)
print("# Código para exportar outliers:")
print("outliers_to_export = outliers_df.copy()")
print("# outliers_to_export.to_csv('outliers_analysis.csv', index=False)")
print(f"# Total registros a exportar: {len(outliers_df)}")

🔍 ANÁLISIS DE OUTLIERS CON MODEL_RESULTS
📊 Datos disponibles:
   • Predicciones: 74 casos
   • Valores reales: 74 casos

🎯 DETECCIÓN DE OUTLIERS:
   • Threshold: $27,528.46
   • Outliers encontrados: 3 casos
   • Porcentaje: 4.05%

🔴 TOP 15 OUTLIERS MÁS PROBLEMÁTICOS
Rank  Index   Salario Real   Predicción    Error Abs    Error %
---------------------------------------------------------------------------
 1      42   $   250000   $   141914   $  108086     43.2%
 2      16   $    90000   $    54140   $   35860     39.8%
 3       6   $   160000   $   130736   $   29264     18.3%

📈 COMPARACIÓN: OUTLIERS vs CASOS NORMALES
Métrica                 Outliers      Normales    Diferencia
-----------------------------------------------------------------
Media Salario          $   166667 $    95282   $   +71385
Std Salario            $    65490 $    47329   $   +18160
Error Promedio         $    57737 $     7894   $   +49842
Error Mediano          $    35860 $     6170   $   +29689

📊 DISTRIBUCI

****************************************************************************

In [ ]:
with open("../../../modelos/salary_with_stats.pkl", "rb") as f:
    print(f.read(50))

b'\x80\x04\x95\xf3\x01\x00\x00\x00\x00\x00\x00}\x94(\x8c\x05model\x94\x8c\x10lightgbm.sklearn\x94\x8c\rLGBMReg'


Próximos pasos:
    Probar un ensamble de modelos
    Implementar monitoreo de drift
    

In [ ]:
! pip freeze > requirements.txt

In [ ]:
dataf=data[(data['Salary'] < 90000) & (data['Salary'] > 35000)]

In [ ]:
dataf

,id,Age,Gender,Education_Level,Job_Title,Years_of_Experience,Salary,Description,Exp_group,Age_group
1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,I am a 28-year-old data analyst with a Master'...,Junior,Joven
3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,I am a 36-year-old female Sales Associate with...,Medio,Medio
5,5,29.0,Male,Bachelor's,Marketing Analyst,2.0,55000.0,I am a 29-year-old Marketing Analyst with a Ba...,Junior,Joven
7,7,31.0,Male,Bachelor's,Sales Manager,4.0,80000.0,I am a 31-year-old Sales Manager with a Bachel...,Junior,Medio
8,8,26.0,Female,Bachelor's,Marketing Coordinator,1.0,45000.0,I am a 26-year-old female Marketing Coordinato...,Junior,Joven
...,...,...,...,...,...,...,...,...,...,...
361,363,33.0,Male,Bachelor's,Junior Marketing Specialist,5.0,70000.0,I am a 33-year-old male with a Bachelor's degr...,Junior,Medio
364,366,31.0,Female,Bachelor's,Junior Financial Analyst,3.0,50000.0,I am a 31-year-old female working as a Junior ...,Junior,Medio
367,369,33.0,Male,Bachelor's,Junior Business Analyst,4.0,60000.0,I am a 33-year-old male working as a Junior Bu...,Junior,Medio
368,370,35.0,Female,Bachelor's,Senior Marketing Analyst,8.0,85000.0,As a 35-year-old Senior Marketing Analyst with...,Medio,Medio
